# Making Stacked Raw Data
In this notebook we are going to transform the data so that we don't have features for each timepoint, but instead concatenate them and add a column for the hour from which they were extracted. This can be used for visualisation and for the investigation of dynamics over time. First we load the data:

In [1]:
import pandas as pd
import organoid_prediction_python as opp
import numpy as np

home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"
table_location = home_directory + "tabular_data"

raw_features = pd.read_csv(f"{table_location}/All_Samples_Feature_Space_All_Features_48h,72h,96h.csv").set_index(["Run","Plate","ID"])
raw_features

Unnamed: 0  048h_raw_BF_AreaShape_area  \
Run Plate ID                                            
RR  1     A1            0                51429.057934   
          A2            1                46107.767989   
          A3            2                54482.752732   
          A4            3                46999.688747   
          A5            4                50680.751535   
...                   ...                         ...   
TR  2     H8          762                47634.615388   
          H9          763                45616.455707   
          H10         764                42336.001394   
          H11         765                40484.132024   
          H12         766                44648.948444   

               048h_raw_BF_AreaShape_aspect_ratio  \
Run Plate ID                                        
RR  1     A1                             1.036110   
          A2                             1.018480   
          A3                             1.070369   
          A4                             1.054378   
          A5                             1.067224   
...                                           ...   
TR  2     H8                             1.074280   
          H9                             1.141554   
          H10                            1.017190   
          H11                            1.034860   
          H12                            1.028101   

               048h_raw_BF_AreaShape_axis_major_length  \
Run Plate ID                                             
RR  1     A1                                260.694837   
          A2                                244.753205   
          A3                                272.945953   
          A4                                251.337713   
          A5                                262.580886   
...                                                ...   
TR  2     H8                                255.443622   
          H9                                257.800200   
          H10                               234.372899   
          H11                               231.114030   
          H12                               241.897406   

               048h_raw_BF_AreaShape_axis_minor_length  \
Run Plate ID                                             
RR  1     A1                                251.609209   
          A2                                240.312256   
          A3                                255.001802   
          A4                                238.375438   
          A5                                246.040977   
...                                                ...   
TR  2     H8                                237.781139   
          H9                                225.832628   
          H10                               230.412135   
          H11                               223.328768   
          H12                               235.285546   

               048h_raw_BF_AreaShape_eccentricity  \
Run Plate ID                                        
RR  1     A1                             0.261703   
          A2                             0.189631   
          A3                             0.356599   
          A4                             0.316996   
          A5                             0.349302   
...                                           ...   
TR  2     H8                             0.365387   
          H9                             0.482314   
          H10                            0.183066   
          H11                            0.257365   
          H12                            0.232206   

               048h_raw_BF_AreaShape_feret_diameter_max  \
Run Plate ID                                              
RR  1     A1                                 266.724612   
          A2                                 252.890770   
          A3                                 280.334245   
          A4                                 256.319703   
          A5                 

### Adding 48h Intensity information for comparison (not included in main analysis due to the extremely low signal)

In [2]:
def rename_columns(df,hour,raw_str_prefix,channel_prefix,cellprof_remove = "__72h_brachyury_original"):
    new_keys = [key.replace(cellprof_remove,"") for key in df.keys()]
    mapping = {key:f"{hour}_{raw_str_prefix}_{channel_prefix}_{new_key}" for key,new_key in zip(df.keys(),new_keys)}
    return df.rename(columns = mapping)

data_bf_mask_48 = opp.reform_cellprofiler_table(
    pd.read_csv(f"{table_location}/Cellprofiler_Data/20230609_BRA_intensity_measures_BF_Object_48h.csv"),spit_out=True
).set_index(["Run","Plate","ID"]).drop(["ObjectNumber"],axis=1)

renamed_bf_mask_48 = rename_columns(data_bf_mask_48,"048h","raw","CH__bf_mask")

df_polarisations = [
    pd.read_csv(
        f"{table_location}/{hour}h/straight_morgana_brachyury_polarisation_{hour}h.csv"
    ).set_index(["Run","Plate","ID"]).rename(columns={
        "first_axis_polarisation":f"0{hour}h_str_CH_AreaShape_Bra_MajorAxis_Polarisation",
        "second_axis_polarisation":f"0{hour}h_str_CH_AreaShape_Bra_MinorAxis_Polarisation",
    }) for hour in [48,72,96]
]
df_polarisations[0]

048h_str_CH_AreaShape_Bra_MajorAxis_Polarisation  \
Run Plate ID                                                      
RR  1     A1                                           0.033602   
          A2                                           0.006087   
          A3                                           0.043632   
          A4                                           0.003000   
          A5                                           0.026882   
...                                                         ...   
TR  2     H8                                           0.005869   
          H9                                           0.011629   
          H10                                          0.007411   
          H11                                          0.002839   
          H12                                          0.003848   

               048h_str_CH_AreaShape_Bra_MinorAxis_Polarisation  
Run Plate ID                                                     
RR  1     A1                                           0.009469  
          A2                                           0.002021  
          A3                                           0.003480  
          A4                                           0.005707  
          A5                                           0.017541  
...                                                         ...  
TR  2     H8                                           0.005420  
          H9                                           0.020696  
          H10                                          0.010388  
          H11                                          0.002295  
          H12                                          0.016681  

[768 rows x 2 columns]

In [3]:
all_props = pd.concat(
    [raw_features,renamed_bf_mask_48,df_polarisations[0]],
    axis=1
).dropna(thresh=2)
all_props

Unnamed: 0  048h_raw_BF_AreaShape_area  \
Run Plate ID                                            
RR  1     A1          0.0                51429.057934   
          A2          1.0                46107.767989   
          A3          2.0                54482.752732   
          A4          3.0                46999.688747   
          A5          4.0                50680.751535   
...                   ...                         ...   
TR  2     H8        762.0                47634.615388   
          H9        763.0                45616.455707   
          H10       764.0                42336.001394   
          H11       765.0                40484.132024   
          H12       766.0                44648.948444   

               048h_raw_BF_AreaShape_aspect_ratio  \
Run Plate ID                                        
RR  1     A1                             1.036110   
          A2                             1.018480   
          A3                             1.070369   
          A4                             1.054378   
          A5                             1.067224   
...                                           ...   
TR  2     H8                             1.074280   
          H9                             1.141554   
          H10                            1.017190   
          H11                            1.034860   
          H12                            1.028101   

               048h_raw_BF_AreaShape_axis_major_length  \
Run Plate ID                                             
RR  1     A1                                260.694837   
          A2                                244.753205   
          A3                                272.945953   
          A4                                251.337713   
          A5                                262.580886   
...                                                ...   
TR  2     H8                                255.443622   
          H9                                257.800200   
          H10                               234.372899   
          H11                               231.114030   
          H12                               241.897406   

               048h_raw_BF_AreaShape_axis_minor_length  \
Run Plate ID                                             
RR  1     A1                                251.609209   
          A2                                240.312256   
          A3                                255.001802   
          A4                                238.375438   
          A5                                246.040977   
...                                                ...   
TR  2     H8                                237.781139   
          H9                                225.832628   
          H10                               230.412135   
          H11                               223.328768   
          H12                               235.285546   

               048h_raw_BF_AreaShape_eccentricity  \
Run Plate ID                                        
RR  1     A1                             0.261703   
          A2                             0.189631   
          A3                             0.356599   
          A4                             0.316996   
          A5                             0.349302   
...                                           ...   
TR  2     H8                             0.365387   
          H9                             0.482314   
          H10                            0.183066   
          H11                            0.257365   
          H12                            0.232206   

               048h_raw_BF_AreaShape_feret_diameter_max  \
Run Plate ID                                              
RR  1     A1                                 266.724612   
          A2                                 252.890770   
          A3                                 280.334245   
          A4                                 256.319703   
          A5                 

Using a function implemented in organoid_prediction_python to doi the stacking and saving the result afterwards

In [4]:
stacked_raw_data = opp.stack_time_data(
    all_props.reset_index()
)
stacked_raw_data.to_csv(table_location+"/"+"raw_data_stacked_timepoints.csv")

In [5]:
stacked_raw_data.keys().to_numpy()

array(['raw_BF_AreaShape_area', 'raw_BF_AreaShape_aspect_ratio',
       'raw_BF_AreaShape_axis_major_length',
       'raw_BF_AreaShape_axis_minor_length',
       'raw_BF_AreaShape_eccentricity',
       'raw_BF_AreaShape_feret_diameter_max',
       'raw_BF_AreaShape_perimeter', 'raw_BF_AreaShape_solidity',
       'str_BF_AreaShape_area', 'str_BF_AreaShape_aspect_ratio',
       'str_BF_AreaShape_eccentricity',
       'str_BF_AreaShape_equivalent_diameter', 'str_BF_AreaShape_extent',
       'str_BF_AreaShape_form_factor',
       'str_BF_AreaShape_inertia_tensor_0_0',
       'str_BF_AreaShape_inertia_tensor_0_1',
       'str_BF_AreaShape_inertia_tensor_1_0',
       'str_BF_AreaShape_inertia_tensor_1_1',
       'str_BF_AreaShape_inertia_tensor_eigvals_0',
       'str_BF_AreaShape_inertia_tensor_eigvals_1',
       'str_BF_AreaShape_locoefa_PC_1', 'str_BF_AreaShape_locoefa_PC_2',
       'str_BF_AreaShape_locoefa_PC_3', 'str_BF_AreaShape_locoefa_PC_4',
       'str_BF_AreaShape_locoefa_PC_5', '